In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pyaudio
import sys
import os
import pickle
import torch
import librosa
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torchvision.models as models
from transformers import HubertModel, AutoProcessor
import torchaudio

In [3]:
class LayerFeatureDataset(Dataset):
    def __init__(self, pkl_path):
        data = pickle.load(open(pkl_path, 'rb'))
        self.X = torch.tensor(data['features'], dtype=torch.float32)  
        self.y = torch.tensor(data['labels'],   dtype=torch.long)    
    def __len__(self):
        return len(self.y)
    def __getitem__(self, i):
        return self.X[i], self.y[i]

class LayerWeightedAggregator(nn.Module):
    def __init__(self, num_layers):
        super().__init__()
        self.w = nn.Parameter(torch.ones(num_layers) / num_layers)
    def forward(self, x):
        w = torch.softmax(self.w, dim=0)             
        return (x * w[None, :, None]).sum(dim=1)

class ResNetClassifier(nn.Module):
    def __init__(self, num_layers, hidden_dim, num_classes):
        super().__init__()
        self.agg = LayerWeightedAggregator(num_layers)
        H = W = int(np.sqrt(hidden_dim))
        assert H*W == hidden_dim, "hidden_dim must be square"
        self.H, self.W = H, W

        self.resnet = models.resnet18(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(1,
            self.resnet.conv1.out_channels,
            kernel_size=self.resnet.conv1.kernel_size,
            stride=self.resnet.conv1.stride,
            padding=self.resnet.conv1.padding,
            bias=False)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        x = self.agg(x)               
        b = x.size(0)
        x = x.view(b, 1, self.H, self.W)
        return self.resnet(x)


In [4]:
pkl_path = "../Features/Opt_Features/ASV_Opt_Dataset.pkl"
dataset = LayerFeatureDataset(pkl_path)
processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_layers  = dataset.X.size(1)
hidden_dim  = dataset.X.size(2)
num_classes = len(torch.unique(dataset.y))

model = ResNetClassifier(num_layers, hidden_dim, num_classes)
model.load_state_dict(torch.load("../model.pth"))
model.eval()
hub = HubertModel.from_pretrained("facebook/hubert-large-ls960-ft", output_hidden_states=True).to(device)
hub.eval()
print("Model loaded successfully")

with open("../ASV_sample.pkl", "rb") as f:
    data = pickle.load(f)

print("Sampled Files:", data["files"])
print("Sampled Labels:", data["labels"])

audio_files = data["files"]
audio_labels = data["labels"]

/home/g7/Desktop/AML/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/g7/Desktop/AML/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model loaded successfully
Sampled Files: ['../ASV/Data/LA_E_6189894.flac', '../ASV/Artifact/LA_E_9638498_filter.wav', '../ASV/Artifact/LA_E_1096928_filter.wav', '../ASV/Data/LA_E_6346279.flac', '../ASV/Artifact/LA_E_5241439_clipping.wav', '../ASV/Artifact/LA_E_2091174_clipping.wav', '../ASV/Data/LA_E_5064331.flac', '../ASV/Artifact/LA_E_5960975_compression.wav', '../ASV/Data/LA_E_8816429.flac', '../ASV/Artifact/LA_E_5167557_clipping.wav']
Sampled Labels: [1, 0, 0, 1, 0, 1, 0, 1, 1, 0]


In [5]:
def load_audio(filename):
    waveform, sample_rate = torchaudio.load(filename)
    waveform = waveform.to(device)
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(sample_rate, 16000)(waveform)
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    return waveform.squeeze(0)


def extract_features(waveform):
    with torch.no_grad():
        inputs = processor(waveform, sampling_rate=16000, return_tensors="pt").to(device)
        outputs = hub(**inputs)
        hidden_states = outputs.hidden_states[2:6]  # layers 3rd to 6th
        pooled = [hs.mean(dim=1).squeeze(0).cpu().numpy() for hs in hidden_states]
        features = np.stack(pooled, axis=0)  
        features = torch.tensor(features, dtype=torch.float32)
    return features

sys.stderr = open(os.devnull, 'w')
def play_audio(file_path):
    try:
        waveform, sample_rate = torchaudio.load(file_path)  
        data = waveform.cpu().numpy().astype(np.float32)

        p = pyaudio.PyAudio()
        stream = p.open(
            format=pyaudio.paFloat32,
            channels=data.shape[0],
            rate=sample_rate,
            output=True
        )

        interleaved = data.T.reshape(-1).tobytes()
        stream.write(interleaved)

        stream.stop_stream()
        stream.close()
        p.terminate()

    except Exception as e:
        print(f"Error playing audio: {e}")

In [6]:
class FakeAudioApp:
    def __init__(self, master):
        self.master = master
        master.title("Fake Audio Detector")
        master.geometry("800x500")
        master.configure(bg="#f9f9f9")

        self.labels = data["labels"]

        style = ttk.Style(master)
        style.theme_use('clam')
        style.configure('TLabel', background='#f9f9f9', font=('Helvetica', 12))
        style.configure('Header.TLabel', font=('Helvetica', 18, 'bold'))
        style.configure('TButton', font=('Helvetica', 12), padding=6)

        header = ttk.Label(master, text="Fake Audio Classification", style='Header.TLabel')
        header.pack(pady=(20, 5))

        controls = ttk.Frame(master, padding=10)
        controls.pack(fill='x')

        ttk.Label(controls, text="Choose sample (1–10):").grid(row=0, column=0, sticky='w')
        self.number_var = tk.StringVar(value="1")
        number_menu = ttk.Combobox(controls, textvariable=self.number_var,
                                   values=[str(i) for i in range(1, 11)],
                                   state='readonly', width=5, font=('Helvetica', 12))
        number_menu.grid(row=0, column=1, padx=10)

        btn_frame = ttk.Frame(master, padding=(10, 10))
        btn_frame.pack()

        self.play_btn = ttk.Button(btn_frame, text="▶ Play", command=self.play_selected, width=20)
        self.play_btn.grid(row=0, column=0, padx=20)

        self.classify_btn = ttk.Button(btn_frame, text="✔ Classify", command=self.classify_selected, width=20)
        self.classify_btn.grid(row=0, column=1, padx=20)

        ttk.Separator(master, orient='horizontal').pack(fill='x', pady=15)

        result_frame = ttk.Frame(master, padding=20)
        result_frame.pack()

        ttk.Label(result_frame, text="Prediction:", font=('Helvetica', 14)).grid(row=0, column=0, sticky='w', padx=10)
        self.result_lbl = ttk.Label(result_frame, text="–", font=('Helvetica', 24, 'bold'), foreground='#007acc')
        self.result_lbl.grid(row=1, column=0, padx=10)

        ttk.Label(result_frame, text="Ground Truth:", font=('Helvetica', 14)).grid(row=0, column=1, sticky='w', padx=40)
        self.ground_lbl = ttk.Label(result_frame, text="–", font=('Helvetica', 24, 'bold'), foreground='#444')
        self.ground_lbl.grid(row=1, column=1, padx=40)

        self.selected_file = None

    def play_selected(self):
        try:
            idx = int(self.number_var.get()) - 1
            self.selected_file = audio_files[idx]
            play_audio(self.selected_file)
        except Exception as e:
            print("Playback Error", str(e))

    def classify_selected(self):
        try:
            idx = int(self.number_var.get()) - 1
            self.selected_file = audio_files[idx]
            waveform = load_audio(self.selected_file)
            if waveform is None:
                return

            features = extract_features(waveform)

            with torch.no_grad():
                outputs = model(features)
                pred = torch.argmax(outputs, dim=1).item()
                label = "Fake" if pred == 1 else "Real"

            self.result_lbl.config(text=label)

            gt = self.labels[idx]
            gt_label = "Fake" if gt == 1 else "Real"
            self.ground_lbl.config(text=gt_label)

        except Exception as e:
            print("Classification Error", str(e))


In [8]:
if __name__ == "__main__":
    root = tk.Tk()
    app = FakeAudioApp(root)
    root.mainloop()